<a href="https://colab.research.google.com/github/jpsiegel/Projects/blob/master/Tarea2_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tarea 2 Jan P. Siegel - Deep Learning

In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=bbfcdf5e7aecab7e94ddfbb88233e29eb494bad7d4d5f77ba3db5c2555be7c37
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [ ]:
GPUs = GPU.getGPUs()
gpu = GPUs[0]  # Only one GPU on Colab and not guaranteed

def printm():
  process = psutil.Process(os.getpid())
  print("RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " | Used: " + humanize.naturalsize(process.memory_info().rss))
  print("VRAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

RAM Free: 12.8 GB  | Used: 118.5 MB
VRAM Free: 15109MB | Used: 0MB | Util   0% Total 15109MB


##Parte 1


###Actividad 1

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

sms_data_path = '/content/drive/MyDrive/DL/T2/SMSSpamCollection'

Mounted at /content/drive


In [ ]:
!pip install d2l

In [ ]:
from string import punctuation
from d2l import torch as d2l

#  Reads sms data file
raw_data = open(sms_data_path, "r")
strings = raw_data.readlines()

vocab_freq = {} # word : count

processed_data = []
processed_spam = []
processed_nospam = []
for s in strings:
  s = s.strip("\n")
  listed = s.split("\t")
  label = 1 if listed[0] == "spam" else 0 # 1 is spam and 0 is not spam
  phrase = listed[1].lower()
  phrase = "".join([c for c in phrase if c not in punctuation]) # eliminate punctuation
  words = phrase.split(" ")
  for word in words:
    try:
      vocab_freq[word] += 1
    except KeyError:
      vocab_freq[word] = 1
  processed_data.append((label, words))
  if label == 1:
    processed_spam.append((label, words))
  else:
    processed_nospam.append((label, words))

vocab = vocab_freq.keys()
print(processed_data[:3]) # (label, lists_of_words)
print("Vocab length:", len(vocab)) # vocabulary of 9662 words
print("Spamm data:", len(processed_spam))
print("No Spamm data:", len(processed_nospam))

[(0, ['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']), (0, ['ok', 'lar', 'joking', 'wif', 'u', 'oni']), (1, ['free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkts', '21st', 'may', '2005', 'text', 'fa', 'to', '87121', 'to', 'receive', 'entry', 'questionstd', 'txt', 'ratetcs', 'apply', '08452810075over18s'])]
Vocab length: 9662
Spamm data: 747
No Spamm data: 4827


In [ ]:
embedding_dim = "100" # dimensiones aceptadas 50, 100, 200, 300 (act 2 y 5)
glove_embedding = d2l.TokenEmbedding(f'glove.6b.{embedding_dim}d') 

In [ ]:
embeds = glove_embedding[vocab] # generate embedding for every word in vocabulary 
print(embeds.shape) # we have 9662 vectors of 100 dims

torch.Size([9662, 100])


La idea de implementar un embedding para las palabras es poder pasar de datos en texto a tensores vectoriales numéricos, esto permite expresar las palabras en un formato que les añada riqueza en su descripción semántica (haciendo que palabras similares tengan valores cercanos) y además las haga consumibles por un modelo de aprendizaje automático.

En particular, GloVe no captura solamente las características estadísticas locales como Word2Vec, sino también las (Glo)bales para crear el (Ve)ctor deseado. Para lograr esto,  toma en cuenta relaciones semánticas locales, pero también deriva relaciones semánticas a partir de una matriz de co-ocurrencia, donde se indica qué tan probable es que una palabra aparezca en el contexto de otra. La gracia de GloVe es que puede expresar esta relación en un vector limpiamente con escasa pérdida de información.

###Actividad 2

In [ ]:
import torch
import torch.nn as nn


class RNN(nn.Module):
  
  def __init__(self, input_dim, hidden_dim, num_layers):
    super().__init__() #input_dim es la dimension del x_t, es la dimension que te deja el embedding (ej word2vec)
    self.hidden_dim = hidden_dim # dimensiones de h_t
    self.num_layers = num_layers # cantidad de capas (filas) ocultas
    self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
    self.linear_out = nn.Linear(hidden_dim, 1) # creo que esto es y
    
  # This method defines the forward pass of the RNN
  def forward(self, input):
    batch_size, _ = input.size()

    # Initializing hidden state for first input
    h0 = self.init_hidden(batch_size)
    # Passing in the input and hidden state to obtain output
    _, hidden_state = self.rnn(input.unsqueeze(2), h0)
    out = self.linear_out(hidden_state.squeeze())
    return out
    
  # This method generates the first hidden state of zeros for the forward pass
  # This creates a tensor of zeros in the shape of our hidden states.
  def init_hidden(self, batch_size):
    hidden = torch.zeros(self.num_layers, batch_size, self.hidden_dim)
    return hidden

##Referencias
https://www.kaggle.com/anindya2906/glove6b